### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## Local Government Area Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average maths score 

* Calculate the average reading score

* Calculate the percentage of students with a passing maths score (50 or greater)

* Calculate the percentage of students with a passing reading score (50 or greater)

* Calculate the percentage of students who passed maths **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [5]:
# Calculate the various elemnts required for the dataframe

school_count = school_data['School ID'].count()
student_count = student_data['Student ID'].count()
total_budget = school_data["budget"].sum()
maths_avg = student_data["maths_score"].mean()
read_avg = student_data["reading_score"].mean()
maths_passperc = student_data[student_data["maths_score"] >= 50]["maths_score"].count()/student_count
read_passperc = student_data[student_data["reading_score"] >= 50]["reading_score"].count()/student_count
overall_passperc= student_data[(student_data["maths_score"] >= 50) & (student_data["reading_score"] >= 50)]["Student ID"].count()/student_count

# Create a dataframe to hold the above results
areasummary = pd.DataFrame({"Total Schools": [school_count],
                            "Total Students": [student_count],
                            "Total Budget":[total_budget],
                            "Average Maths Score":[maths_avg],
                            "Average Reading Score":[read_avg],
                            "% Passing Maths":[maths_passperc],
                            "% Passing Reading":[read_passperc],
                            "% Overall Passing":[overall_passperc]})

# Optional: give the displayed data cleaner formatting
areasummary.style.format({"Total Students": "{:,.0f}",
                        "Total Budget": "${:,.0f}",
                       "Average Maths Score": "{:.2f}",
                       "Average Reading Score": "{:.2f}", 
                       "% Passing Maths": "{:.2%}", 
                       "% Passing Reading": "{:.2%}", 
                       "% Overall Passing": "{:.2%}"})


,Total Schools,Total Students,Total Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428",70.34,69.98,86.08%,84.43%,72.81%


## School Summary

* Create an overview table that summarises key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Maths Score
  * Average Reading Score
  * % Passing Maths
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed maths **and** reading.)
  
* Create a dataframe to hold the above results

In [6]:
# create groups of data per school
group_by_school = school_data_complete.set_index('school_name').groupby(['school_name'])

# Column Headings required (grouped by School) are:
# School Type	Total Students	Total School Budget	Per Student Budget	Average Maths Score	
# Average Reading Score	% Passing Maths	% Passing Reading	% Overall Passing

# The calculations of each column are:
school_types = school_data.set_index('school_name')['type']
students_ps=group_by_school["Student ID"].count()
budget_pschool=school_data.set_index('school_name')['budget']
budget_pstudent=budget_pschool/students_ps
maths_avg_pschool =group_by_school["maths_score"].mean()
reading_avg_pschool =group_by_school["reading_score"].mean()

# use complete sets as grouped by does not allow >
pass_maths_ps = school_data_complete[school_data_complete["maths_score"] >= 50]
pass_maths_ps = pass_maths_ps.groupby("school_name")["Student ID"].count()
pass_maths_perc_pschool = pass_maths_ps/students_ps

pass_reading_ps=school_data_complete[school_data_complete["reading_score"] >= 50]
pass_reading_ps = pass_reading_ps.groupby("school_name")["Student ID"].count()
pass_reading_perc_pschool=pass_reading_ps/students_ps

overall_pass_total_ps = student_data[(student_data['reading_score']>=50) & (student_data['maths_score']>=50)]
overall_pass_total_ps = overall_pass_total_ps.groupby("school_name")["Student ID"].count()
overall_pass_total_ps_perc=overall_pass_total_ps/students_ps

# create school_summary dataframe
school_summary = pd.DataFrame({"School Type": school_types,
                            "Total Students": students_ps,
                            "Per Student Budget": budget_pstudent,
                            "Total School Budget": budget_pschool,
                            "Average Maths Score": maths_avg_pschool,
                            "Average Reading Score": reading_avg_pschool,
                            '% Passing Maths': pass_maths_perc_pschool,
                            '% Passing Reading': pass_reading_perc_pschool,
                            "% Overall Passing": overall_pass_total_ps_perc})
school_summary

# clean format of dataframe output
school_summary.style.format({"Total Students": "{:,.0f}",
                        "Total School Budget": "${:,.0f}",
                        "Per Student Budget": "${:,.0f}",
                       "Average Maths Score": "{:.2f}",
                       "Average Reading Score": "{:.2f}",
                       "% Passing Maths": "{:.2%}", 
                       "% Passing Reading": "{:.2%}", 
                       "% Overall Passing": "{:.2%}"})

,School Type,Total Students,Per Student Budget,Total School Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Bailey High School,Government,"4,976",$628,"$3,124,928",72.35,71.01,91.64%,87.38%,80.08%
Cabrera High School,Independent,"1,858",$582,"$1,081,356",71.66,71.36,90.85%,89.07%,80.79%
Figueroa High School,Government,"2,949",$639,"$1,884,411",68.70,69.08,81.65%,82.81%,67.65%
Ford High School,Government,"2,739",$644,"$1,763,916",69.09,69.57,82.44%,82.22%,67.47%
Griffin High School,Independent,"1,468",$625,"$917,500",71.79,71.25,91.21%,88.49%,81.34%
Hernandez High School,Government,"4,635",$652,"$3,022,020",68.87,69.19,80.95%,81.88%,66.36%
Holden High School,Independent,427,$581,"$248,087",72.58,71.66,89.93%,88.52%,78.92%
Huang High School,Government,"2,917",$655,"$1,910,635",68.94,68.91,81.69%,81.45%,66.71%
Johnson High School,Government,"4,761",$650,"$3,094,650",68.84,69.04,82.06%,81.98%,67.19%
Pena High School,Independent,962,$609,"$585,858",72.09,71.61,91.68%,86.59%,79.21%


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

* Sort and display the five worst-performing schools by % overall passing.

In [7]:
bottom_schools = school_summary.sort_values(by='% Overall Passing', ascending=True).head(5)

bottom_schools.style.format({"Total Students": "{:,.0f}",
                        "Total School Budget": "${:,.0f}",
                        "Per Student Budget": "${:,.0f}",
                       "Average Maths Score": "{:.2f}",
                       "Average Reading Score": "{:.2f}",
                       "% Passing Maths": "{:.2%}", 
                       "% Passing Reading": "{:.2%}", 
                       "% Overall Passing": "{:.2%}"})


,School Type,Total Students,Per Student Budget,Total School Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Hernandez High School,Government,"4,635",$652,"$3,022,020",68.87,69.19,80.95%,81.88%,66.36%
Huang High School,Government,"2,917",$655,"$1,910,635",68.94,68.91,81.69%,81.45%,66.71%
Johnson High School,Government,"4,761",$650,"$3,094,650",68.84,69.04,82.06%,81.98%,67.19%
Wilson High School,Independent,"2,283",$578,"$1,319,574",69.17,68.88,82.79%,81.30%,67.46%
Ford High School,Government,"2,739",$644,"$1,763,916",69.09,69.57,82.44%,82.22%,67.47%


## Maths Scores by Year

* Create a table that lists the average maths score for students of each year level (9, 10, 11, 12) at each school.

  * Create a pandas series for each year. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [8]:
# define the datasets for the various years
year_9= school_data_complete[(school_data_complete["year"] == 9)]
year_10 = school_data_complete[(school_data_complete["year"] == 10)]
year_11 = school_data_complete[(school_data_complete["year"] == 11)]
year_12 = school_data_complete[(school_data_complete["year"] == 12)]

# calculate the average maths scores per year
year_9_grp_avg_math = year_9.groupby(['school_name'])['maths_score'].mean()
year_10_grp_avg_math = year_10.groupby(['school_name'])['maths_score'].mean()
year_11_grp_avg_math = year_11.groupby(['school_name'])['maths_score'].mean()
year_12_grp_avg_math = year_12.groupby(['school_name'])['maths_score'].mean()

# create dataframe 
maths_by_year =pd.DataFrame({"Year 9": year_9_grp_avg_math, 
                            "Year 10": year_10_grp_avg_math,
                            "Year 11": year_11_grp_avg_math,
                            "Year 12": year_12_grp_avg_math})

# clean format dataframe
maths_by_year.style.format({"Year 9": "{:,.2f}",
                        "Year 10": "{:,.2f}",
                        "Year 11": "{:,.2f}",
                        "Year 12": "{:.2f}",})


,Year 9,Year 10,Year 11,Year 12
school_name,,,,
Bailey High School,72.49,71.90,72.37,72.68
Cabrera High School,72.32,72.44,71.01,70.60
Figueroa High School,68.48,68.33,68.81,69.33
Ford High School,69.02,69.39,69.25,68.62
Griffin High School,72.79,71.09,71.69,71.47
Hernandez High School,68.59,68.87,69.15,68.99
Holden High School,70.54,75.11,71.64,73.41
Huang High School,69.08,68.53,69.43,68.64
Johnson High School,69.47,67.99,68.64,69.29


## Reading Score by Year

* Perform the same operations as above for reading scores

In [9]:
# use the datasets for the various years from previous maths score by year

# calculate the average maths scores per year
year_9_grp_avg_reading = year_9.groupby(['school_name'])['reading_score'].mean()
year_10_grp_avg_reading = year_10.groupby(['school_name'])['reading_score'].mean()
year_11_grp_avg_reading = year_11.groupby(['school_name'])['reading_score'].mean()
year_12_grp_avg_reading = year_12.groupby(['school_name'])['reading_score'].mean()

# create dataframe 
reading_by_year =pd.DataFrame({"Year 9": year_9_grp_avg_reading, 
                            "Year 10": year_10_grp_avg_reading,
                            "Year 11": year_11_grp_avg_reading,
                            "Year 12": year_12_grp_avg_reading})

# clean format dataframe
reading_by_year.style.format({"Year 9": "{:,.2f}",
                        "Year 10": "{:,.2f}",
                        "Year 11": "{:,.2f}",
                        "Year 12": "{:.2f}",})

,Year 9,Year 10,Year 11,Year 12
school_name,,,,
Bailey High School,70.90,70.85,70.32,72.20
Cabrera High School,71.17,71.33,71.20,71.86
Figueroa High School,70.26,67.68,69.15,69.08
Ford High School,69.62,68.99,70.74,68.85
Griffin High School,72.03,70.75,72.39,69.43
Hernandez High School,68.48,70.62,68.42,69.24
Holden High School,71.60,71.10,73.31,70.48
Huang High School,68.67,69.52,68.74,68.67
Johnson High School,68.72,69.30,69.97,67.99


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Maths Score
  * Average Reading Score
  * % Passing Maths
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [10]:
spending_summary=school_summary.copy()
spending_bins=[0,585,630,645,680]
labels=["< $585","$585-630","$630-645", "$645-680"]
spending_summary["Spending Ranges (Per Student)"]=pd.cut(spending_summary['Per Student Budget'], spending_bins, labels=labels)

spending_maths_scores = spending_summary.groupby(["Spending Ranges (Per Student)"]).mean()["Average Maths Score"]
spending_reading_scores = spending_summary.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]
spending_passing_maths = spending_summary.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Maths"]
spending_passing_reading = spending_summary.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]
overall_passing_spending = spending_summary.groupby(["Spending Ranges (Per Student)"]).mean()["% Overall Passing"]

spending_summary = pd.DataFrame({"Average Maths Score": spending_maths_scores, "Average Reading Score": spending_reading_scores,
                                	"% Passing Maths": spending_passing_maths,"% Passing Reading": spending_passing_reading, 
                                    "% Overall Passing": overall_passing_spending})

spending_summary.style.format({"Average Maths Score": "{:.2f}",
                       "Average Reading Score": "{:.2f}",
                       "% Passing Maths": "{:.2%}", 
                       "% Passing Reading": "{:.2%}", 
                       "% Overall Passing": "{:.2%}"})

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
< $585,71.36,70.72,88.84%,86.39%,76.72%
$585-630,72.07,71.03,91.52%,87.29%,79.88%
$630-645,69.85,69.84,84.69%,83.76%,71.00%
$645-680,68.88,69.05,81.57%,81.77%,66.76%


## Scores by School Size

* Perform the same operations as above, based on school size.

In [11]:
size_summary=school_summary.copy()
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

size_summary["School Size"]=pd.cut(size_summary['Total Students'], size_bins, labels=labels)

spending_maths_scores = size_summary.groupby(["School Size"]).mean()["Average Maths Score"]
spending_reading_scores = size_summary.groupby(["School Size"]).mean()["Average Reading Score"]
spending_passing_maths = size_summary.groupby(["School Size"]).mean()["% Passing Maths"]
spending_passing_reading = size_summary.groupby(["School Size"]).mean()["% Passing Reading"]
overall_passing_spending = size_summary.groupby(["School Size"]).mean()["% Overall Passing"]

size_summary = pd.DataFrame({"Average Maths Score": spending_maths_scores, "Average Reading Score": spending_reading_scores,
                                	"% Passing Maths": spending_passing_maths,"% Passing Reading": spending_passing_reading, 
                                    "% Overall Passing": overall_passing_spending})

size_summary.style.format({"Average Maths Score": "{:.2f}",
                       "Average Reading Score": "{:.2f}",
                       "% Passing Maths": "{:.2%}", 
                       "% Passing Reading": "{:.2%}", 
                       "% Overall Passing": "{:.2%}"})

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),72.34,71.64,90.81%,87.56%,79.07%
Medium (1000-2000),71.42,70.72,89.85%,86.71%,78.04%
Large (2000-5000),69.75,69.58,84.25%,83.30%,70.29%


## Scores by School Type

* Perform the same operations as above, based on school type

In [12]:

# calculate elemnts for dataframe
maths_score_pt=school_summary.groupby(['School Type']).mean()['Average Maths Score']
reading_score_pt=school_summary.groupby(['School Type']).mean()['Average Maths Score']
maths_pass_perc_pt=school_summary.groupby(['School Type']).mean()['% Passing Maths']
reading_pass_perc_pt=school_summary.groupby(['School Type']).mean()['% Passing Reading']
Overall_pass_perc_pt=school_summary.groupby(['School Type']).mean()['% Overall Passing']

# create dataframe
schooltype_summary= pd.DataFrame({"Average Maths Score": maths_score_pt, "Average Reading Score": reading_score_pt,
                                	"% Passing Maths": maths_pass_perc_pt,"% Passing Reading": reading_pass_perc_pt, 
                                    "% Overall Passing": Overall_pass_perc_pt})

# clean format of dataframe
schooltype_summary.style.format({"Average Maths Score": "{:.2f}",
                       "Average Reading Score": "{:.2f}",
                       "% Passing Maths": "{:.2%}", 
                       "% Passing Reading": "{:.2%}", 
                       "% Overall Passing": "{:.2%}"})

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Type,,,,,
Government,69.83,69.83,84.46%,83.59%,70.70%
Independent,71.37,71.37,89.20%,86.25%,76.97%
